In [39]:
print("ok")

ok


In [40]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [41]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [42]:
extracted_data = load_pdf("data/")


In [43]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500 , chunk_overlap=20)
    text_chunks= text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks =text_split(extracted_data)
text_chunks


[Document(page_content='Journal of Physics: Conference Series\nPAPER • OPEN ACCESS\nVideo Content-Based Advertisement Recommendation System using\nClassification Technique of Machine Learning\nTo cite this article: R C Konapure and L M R J Lobo 2021 J. Phys.: Conf. Ser. 1854 012025\n\xa0\nView the article online  for updates and enhancements. \n \nThis content was downloaded from IP address 45.114.159.106 on 11/05/2021 at 09:36', metadata={'source': 'data\\doc.pdf', 'page': 0}),
 Document(page_content='Content from this work may be used under the terms of the Creative Commons Attribution 3.0 licence. Any further distribution\nof this work must maintain attribution to the author(s) and the title of the work, journal citation and DOI.\nPublished under licence by IOP Publishing LtdFEST 2020\nJournal of Physics: Conference Series 1854 (2021) 012025IOP Publishing\ndoi:10.1088/1742-6596/1854/1/012025\n1 \n   \n \n \nVideo Content-Based Advertisement Recommendation System', metadata={'source'

In [44]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [56]:
embeddings = download_hugging_face_embeddings()
embeddings

c:\Users\Bhupesh\Anaconda3\envs\chatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [46]:
import os
from pinecone import Pinecone, ServerlessSpec
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [47]:
index_name = "chatbot"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)
index

In [57]:
from langchain_pinecone import PineconeVectorStore  
docsearch = PineconeVectorStore.from_texts(
        [t.page_content for t in text_chunks],
        index_name=index_name,
        embedding=embeddings
    )

docsearch

In [58]:
docsearch=PineconeVectorStore.from_existing_index(index_name, embeddings)



docs=docsearch.similarity_search(query, k=2)

print("Result", docs)

Result [Document(page_content='arXiv:1909.04190v1 [cs.IR]   9 Sep 2019 Recommendation System-based Upper Confidence \nBound for Online Advertising. \n[8] Missi Hikmatyar and Ruuhwan. Journal of Physics: Conference Series 1477 032024  2020 Book \nRecommendation System Development Using User-Based Collaborative Filtering. \n[9] Thirumalaisamy R, Sudheer K, Battulab V. International Journal of Computer Science \nProcedia 50 Advertisement Posting based on Consumer Behavior. 329 – 334'), Document(page_content='arXiv:1909.04190v1 [cs.IR]   9 Sep 2019 Recommendation System-based Upper Confidence \nBound for Online Advertising. \n[8] Missi Hikmatyar and Ruuhwan. Journal of Physics: Conference Series 1477 032024  2020 Book \nRecommendation System Development Using User-Based Collaborative Filtering. \n[9] Thirumalaisamy R, Sudheer K, Battulab V. International Journal of Computer Science \nProcedia 50 Advertisement Posting based on Consumer Behavior. 329 – 334')]


In [50]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [51]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}


In [52]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})




In [54]:
# retriever = docsearch.as_retriever(search_kwargs={"k": 2})

# from langchain.chains import RetrievalQA

# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,  # Your language model instance
#     chain_type="stuff",  # The chain type (could be "map_reduce", "stuff", etc.)
#     retriever=retriever,  # The retriever instance (e.g., from Pinecone or other vector databases)
#     return_source_documents=True,  # Set to True if you want to return the source documents
#     chain_type_kwargs=chain_type_kwargs  # Additional chain-specific keyword arguments
# )



In [61]:
from langchain.chains.question_answering import load_qa_chain
retriever = docsearch.as_retriever(search_kwargs={"k": 2})
chain = load_qa_chain(llm,chain_type="stuff")


In [62]:
query = "What is system?"
docs = retriever.get_relevant_documents(query)  # Get relevant documents from the retriever
result = chain.run(input_documents=docs, question=query)  # Run the QA chain with the retrieved documents

print(result)

 System in this context refers to a well-established framework or structure for collecting, organizing, and analyzing data. The text mentions that the established system is inherently based on collecting and estimating information on targeted variables, which suggests that it is a pre-existing methodology for gathering and analyzing data. Therefore, the answer to the question "What is system?" in this context is a well-defined framework or structure for data collection and analysis.


In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])